<a href="https://colab.research.google.com/github/elifdonmez/disaster_tweet_analysis/blob/main/Disaster_Tweet_Analysis_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries

In [ ]:
pip install transformers pandas torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

1.   Convert the sample data to a DataFrame and standardize the date format.
2.   Remove Duplicates
3.   Remove Retweets
4.   Relevance Keywords, ignore the tweets that has not the keywords in the keyword list.
5.   Classify Relevance and put the help requests into class 3
6.   Apply the relevance classification function and filter the DataFrame to keep only relevant tweets.

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch

# Sample tweet data (replace this with your actual data)
data = {
    'tweet': [
        "Help needed after the earthquake in the city.",
        "Just had a great day at the park!",
        "RT @user: Floods are devastating the region!",
        "Wildfires are spreading rapidly. #disaster",
        "This is a duplicate tweet.",
        "This is a duplicate tweet.",
        "We are safe now after the hurricane.",
        "Looking for a place to eat tonight."
    ],
    'date': [
        "2024-05-01", "2024-05-10", "2024-05-20", "2024-05-25",
        "2024-05-25", "2024-05-25", "2024-05-26", "2024-05-26"
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])

# Remove duplicates
df = df.drop_duplicates()

# Remove retweets
df = df[~df['tweet'].str.startswith('RT')]

# Keywords for classification
help_keywords = ["help", "needed", "require"]
disaster_keywords = ["droughts", "floods", "wildfires", "hurricanes", "earthquakes", "tornadoes", "aftershock", "hazard", "safe"]

# Load BERT model and tokenizer for sentiment analysis
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Function to classify tweet category
def classify_category(tweet):
    tweet_lower = tweet.lower()
    # Check for help request keywords
    if any(keyword in tweet_lower for keyword in help_keywords):
        return "Help Request"

    # Check for disaster status information keywords
    if any(keyword in tweet_lower for keyword in disaster_keywords):
        return "Disaster Status Information"

    return "Irrelevant"

# Apply classification function
df['category'] = df['tweet'].apply(classify_category)

# Filter relevant categories
relevant_categories = ["Help Request", "Disaster Status Information"]
df_relevant = df[df['category'].isin(relevant_categories)]

# Function to get sentiment
def get_sentiment(tweet):
    result = sentiment_analysis(tweet)
    star_rating = int(result[0]['label'].split()[0])

    if star_rating == 1:
        return "Very Negative"
    elif star_rating == 2:
        return "Negative"
    elif star_rating == 3:
        return "Neutral"
    elif star_rating == 4:
        return "Positive"
    elif star_rating == 5:
        return "Very Positive"

# Apply sentiment analysis to relevant tweets
df_relevant['sentiment'] = df_relevant['tweet'].apply(get_sentiment)

# Display the cleaned, filtered, and analyzed DataFrame
print(df_relevant)


                                           tweet       date  \
0  Help needed after the earthquake in the city. 2024-05-01   
3     Wildfires are spreading rapidly. #disaster 2024-05-25   
6           We are safe now after the hurricane. 2024-05-26   

                      category      sentiment  
0                 Help Request       Positive  
3  Disaster Status Information  Very Negative  
6  Disaster Status Information  Very Positive  


<ipython-input-11-d890d9f3f37f>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['sentiment'] = df_relevant['tweet'].apply(get_sentiment)
